# **K-Nearest Neighbors** 

In [35]:
import numpy as np
import pandas as pd

In [36]:
df = pd.read_csv('AB_NYC_2019.csv')

In [37]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,10/19/2018,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,5/21/2019,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,5/22/2019,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,7/5/2019,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,11/19/2018,0.10,1,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48895 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48895 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [39]:
df = df.fillna(0)
df = df.set_index('neighbourhood_group')

In [40]:
df.tail(20)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
neighbourhood_group,,,,,,,,,,,,,,,
Manhattan,36478343,Welcome all as family,274273284,Anastasia,East Harlem,40.78749,-73.94749,Private room,140,1,0,5/21/2019,0.0,1,180
Manhattan,36478357,"Cozy, Air-Conditioned Private Bedroom in Harlem",177932088,Joseph,Harlem,40.80953,-73.95410,Private room,60,1,0,5/21/2019,0.0,1,26
Brooklyn,36479230,Studio sized room with beautiful light,65767720,Melanie,Bushwick,40.70418,-73.91471,Private room,42,7,0,5/21/2019,0.0,1,16
Queens,36479723,Room for rest,41326856,Jeerathinan,Elmhurst,40.74477,-73.87727,Private room,45,1,0,5/21/2019,0.0,5,172
Brooklyn,36480292,Gorgeous 1.5 Bdr with a private yard- Williams...,540335,Lee,Williamsburg,40.71728,-73.94394,Entire home/apt,120,20,0,5/21/2019,0.0,1,22
Brooklyn,36481315,The Raccoon Artist Studio in Williamsburg New ...,208514239,Melki,Williamsburg,40.71232,-73.94220,Entire home/apt,120,1,0,5/21/2019,0.0,3,365
Brooklyn,36481615,"Peaceful space in Greenpoint, BK",274298453,Adrien,Greenpoint,40.72585,-73.94001,Private room,54,6,0,5/21/2019,0.0,1,15
Brooklyn,36482231,Bushwick _ Myrtle-Wyckoff,66058896,Luisa,Bushwick,40.69652,-73.91079,Private room,40,20,0,5/21/2019,0.0,1,31
Manhattan,36482416,Sunny Bedroom NYC! Walking to Central Park!!,131529729,Kendall,East Harlem,40.79755,-73.93614,Private room,75,2,0,5/21/2019,0.0,2,364


In [41]:
df_B = pd.DataFrame(df.loc["Brooklyn", ["price","number_of_reviews", "reviews_per_month", "minimum_nights", "availability_365" ]])
df_M = pd.DataFrame(df.loc["Manhattan", ["price","number_of_reviews", "reviews_per_month", "minimum_nights", "availability_365"]])
df_Q = pd.DataFrame(df.loc["Queens", ["price","number_of_reviews", "reviews_per_month", "minimum_nights", "availability_365"]])
df_X = pd.DataFrame(df.loc["Bronx", ["price","number_of_reviews", "reviews_per_month", "minimum_nights", "availability_365"]])
df_S = pd.DataFrame(df.loc["Staten Island", ["price","number_of_reviews", "reviews_per_month", "minimum_nights", "availability_365"]])

# **Manhattan**

In [42]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


def knn_train_test(X, target, df_M):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_M.index)
    rand_df_M = df.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_M) / 2)
    train_df_M = rand_df_M.iloc[0:train_rows]
    test_df_M = rand_df_M.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_M[[X]], train_df_M[target])
    predictions = knn.predict(test_df_M[[X]])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_M[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [43]:
uni_rmse = {}

for i in X:
    if i != 'price':
        uni_rmse[i] = knn_train_test(i, 'price', X)
        
# Create a Series object from the dictionary so we can easily view the results, sort, etc.

uni_rmse = pd.Series(uni_rmse)
uni_rmse.sort_values()

NameError: name 'X' is not defined

In [73]:
def knn_train_test(X, target, df_M):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_M.index)
    rand_df_M = df_M.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_M) / 2)
    train_df_M = rand_df_M.iloc[0:train_rows]
    test_df_M = rand_df_M.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_M[X], train_df_M[target])
    predictions = knn.predict(test_df_M[X])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_M[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse
    

In [74]:
k_rmse_multi_results = {}


two_values = ['reviews_per_month', 'minimum_nights']
rmse_val = knn_train_test(two_values, 'price', df_M)
k_rmse_multi_results["two values"] = rmse_val

three_values = ['reviews_per_month', 'minimum_nights', 'availability_365']
rmse_val = knn_train_test(three_values, 'price', df_M)
k_rmse_multi_results["three values"] = rmse_val

four_values = ['reviews_per_month', 'minimum_nights', 'availability_365', 'number_of_reviews']
rmse_val = knn_train_test(three_values, 'price', df_M)
k_rmse_multi_results["four values"] = rmse_val

k_rmse_multi_results

/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


{'two values': 326.23723606080796,
 'three values': 332.0027757947884,
 'four values': 332.0027757947884}

**Brooklyn**

In [62]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


def knn_train_test(X, target, df_B):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_B.index)
    rand_df_B = df.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_B) / 2)
    train_df_B = rand_df_B.iloc[0:train_rows]
    test_df_B = rand_df_B.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_B[[X]], train_df_B[target])
    predictions = knn.predict(test_df_B[[X]])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_B[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [63]:
uni_rmse = {}

for i in X:
    if i != 'price':
        uni_rmse[i] = knn_train_test(i, 'price', X)
        
# Create a Series object from the dictionary so we can easily view the results, sort, etc.

uni_rmse = pd.Series(uni_rmse)
uni_rmse.sort_values()

/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


reviews_per_month    254.406857
minimum_nights       268.789641
availability_365     313.782051
number_of_reviews    695.737448
dtype: float64

In [64]:
def knn_train_test(X, target, df_B):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_B.index)
    rand_df_B = df_B.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_B) / 2)
    train_df_B = rand_df_B.iloc[0:train_rows]
    test_df_B = rand_df_B.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_B[X], train_df_B[target])
    predictions = knn.predict(test_df_B[X])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_B[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse


In [65]:
k_rmse_multi_results = {}


two_values = ['reviews_per_month', 'minimum_nights']
rmse_val = knn_train_test(two_values, 'price', df_B)
k_rmse_multi_results["two values"] = rmse_val

three_values = ['reviews_per_month', 'minimum_nights', 'availability_365']
rmse_val = knn_train_test(three_values, 'price', df_B)
k_rmse_multi_results["three values"] = rmse_val

four_values = ['reviews_per_month', 'minimum_nights', 'availability_365', 'number_of_reviews']
rmse_val = knn_train_test(three_values, 'price', df_B)
k_rmse_multi_results["four values"] = rmse_val

k_rmse_multi_results

/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


{'two values': 216.28225670236063,
 'three values': 184.39574359153653,
 'four values': 184.39574359153653}

# **Queens**

In [47]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


def knn_train_test(X, target, df_Q):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_Q.index)
    rand_df_Q = df.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_Q) / 2)
    train_df_Q = rand_df_Q.iloc[0:train_rows]
    test_df_Q = rand_df_Q.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_Q[[X]], train_df_Q[target])
    predictions = knn.predict(test_df_Q[[X]])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_Q[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse


In [48]:
uni_rmse = {}

for i in X:
    if i != 'price':
        uni_rmse[i] = knn_train_test(i, 'price', X)
        
# Create a Series object from the dictionary so we can easily view the results, sort, etc.

uni_rmse = pd.Series(uni_rmse)
uni_rmse.sort_values()

NameError: name 'X' is not defined

In [49]:

def knn_train_test(X, target, df_Q):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_Q.index)
    rand_df_Q = df_Q.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_Q) / 2)
    train_df_Q = rand_df_Q.iloc[0:train_rows]
    test_df_Q = rand_df_Q.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_Q[X], train_df_Q[target])
    predictions = knn.predict(test_df_Q[X])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_Q[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse


In [50]:
k_rmse_multi_results = {}


two_values = ['reviews_per_month', 'minimum_nights']
rmse_val = knn_train_test(two_values, 'price', df_Q)
k_rmse_multi_results["two values"] = rmse_val

three_values = ['reviews_per_month', 'minimum_nights', 'availability_365']
rmse_val = knn_train_test(three_values, 'price', df_Q)
k_rmse_multi_results["three values"] = rmse_val

four_values = ['reviews_per_month', 'minimum_nights', 'availability_365', 'number_of_reviews']
rmse_val = knn_train_test(three_values, 'price', df_Q)
k_rmse_multi_results["four values"] = rmse_val

k_rmse_multi_results

/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


{'two values': 120.06368635940541,
 'three values': 106.3330629120917,
 'four values': 106.3330629120917}

# **Bronx**

In [51]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


def knn_train_test(X, target, df_X):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_X.index)
    rand_df_X = df.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_X) / 2)
    train_df_X = rand_df_X.iloc[0:train_rows]
    test_df_X = rand_df_X.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_X[[X]], train_df_X[target])
    predictions = knn.predict(test_df_X[[X]])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_X[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse


In [53]:
uni_rmse = {}

for i in X:
    if i != 'price':
        uni_rmse[i] = knn_train_test(i, 'price', X)
        
# Create a Series object from the dictionary so we can easily view the results, sort, etc.

uni_rmse = pd.Series(uni_rmse)
uni_rmse.sort_values()

NameError: name 'X' is not defined

In [54]:
def knn_train_test(X, target, df_X):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_X.index)
    rand_df_X = df_X.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_X) / 2)
    train_df_X = rand_df_X.iloc[0:train_rows]
    test_df_X = rand_df_X.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_X[X], train_df_X[target])
    predictions = knn.predict(test_df_X[X])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_X[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [55]:
k_rmse_multi_results = {}


two_values = ['reviews_per_month', 'minimum_nights']
rmse_val = knn_train_test(two_values, 'price', df_X)
k_rmse_multi_results["two values"] = rmse_val

three_values = ['reviews_per_month', 'minimum_nights', 'availability_365']
rmse_val = knn_train_test(three_values, 'price', df_X)
k_rmse_multi_results["three values"] = rmse_val

four_values = ['reviews_per_month', 'minimum_nights', 'availability_365', 'number_of_reviews']
rmse_val = knn_train_test(three_values, 'price', df_X)
k_rmse_multi_results["four values"] = rmse_val

k_rmse_multi_results


/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


{'two values': 147.42358025313325,
 'three values': 134.57341773881186,
 'four values': 134.57341773881186}

# **Staten Island**

In [56]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


def knn_train_test(S, target, df_S):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_S.index)
    rand_df_S = df.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_S) / 2)
    train_df_S = rand_df_S.iloc[0:train_rows]
    test_df_S = rand_df_S.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_S[[X]], train_df_S[target])
    predictions = knn.predict(test_df_S[[X]])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_S[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [57]:
uni_rmse = {}

for i in X:
    if i != 'price':
        uni_rmse[i] = knn_train_test(i, 'price', X)
        
# Create a Series object from the dictionary so we can easily view the results, sort, etc.

uni_rmse = pd.Series(uni_rmse)
uni_rmse.sort_values()

NameError: name 'X' is not defined

In [58]:
def knn_train_test(X, target, df_S):
    
    np.random.seed(1)
        
    # Randomize order of rows in data frame.
    shuffled_index = np.random.permutation(df_S.index)
    rand_df_S = df_S.reindex(shuffled_index)

    # Divide number of rows in half and round.
    train_rows = int(len(rand_df_S) / 2)
    train_df_S = rand_df_S.iloc[0:train_rows]
    test_df_S = rand_df_S.iloc[train_rows:]
    
    # Fit a knn model and make predictions
    knn = KNeighborsRegressor()
    knn.fit(train_df_S[X], train_df_S[target])
    predictions = knn.predict(test_df_S[X])
    
    # Calculate the mse and rmse
    mse = mean_squared_error(test_df_S[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [59]:
k_rmse_multi_results = {}


two_values = ['reviews_per_month', 'minimum_nights']
rmse_val = knn_train_test(two_values, 'price', df_S)
k_rmse_multi_results["two values"] = rmse_val

three_values = ['reviews_per_month', 'minimum_nights', 'availability_365']
rmse_val = knn_train_test(three_values, 'price', df_S)
k_rmse_multi_results["three values"] = rmse_val

four_values = ['reviews_per_month', 'minimum_nights', 'availability_365', 'number_of_reviews']
rmse_val = knn_train_test(three_values, 'price', df_S)
k_rmse_multi_results["four values"] = rmse_val

k_rmse_multi_results


/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/usr/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


{'two values': 109.21399980776592,
 'three values': 200.40537420612048,
 'four values': 200.40537420612048}